In [92]:
import os, sys
sys.path.append(os.path.abspath("../"))
from utils.toolbox import *

import joblib
import numpy as np
import pandas as pd
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import FunctionTransformer, OrdinalEncoder

In [93]:
# Se lee el dataset de trabajo
df = pd.read_csv("../data/Datos_Reservas_full.csv")

# Se extrae una muestra aleatoria de mil registros y se guarda el archivo
df_muestra = df.sample(1000, random_state = 42)
df_muestra.to_csv("../data_sample/muestra_reservas.csv", index = False) 

# Se eliminan esos registros del dataset original
df = df[~df.isin(df_muestra).all(axis=1)]

In [94]:
def transform_test(df):
    #---------------------------- CAMBIO 1-----------------------------------------
    # ASIGNACIÓN DE MEDIANA A POMOCIONES Y DEVOLUCIONES
    # ASIGNACIÓN DE MEDIANA A VALORES NULL
    # TRANSFORMACIÓN LOGÍSTICA A LA COLUMNA 'daily_price'
    df.loc[df["daily_price"] <= 0, "daily_price"] = 95.2
    df.loc[df["daily_price"].isnull(), "daily_price"] = 95.2
    df["daily_price"] = np.log10(df["daily_price"])
    #---------------------------- CAMBIO 2-----------------------------------------
    # ASIGNACIÓN DE MEDIANA A VALORES INCONSISTENTES Y TRANSFORMACIÓN LOGÍSTICA A LA COLUMNA 'lead_time'
    df.loc[df["lead_time"] <= 0, "lead_time"] = 66
    df.loc[df["lead_time"].isnull(), "lead_time"] = 66
    df["lead_time"] = np.log10(df["lead_time"])
    #---------------------------- CAMBIO 3-----------------------------------------
    # ASIGNACIÓN DE MEDIANA EN 'adults' DE REGISTROS CON VALOR 0 COINCIDENTES EN LAS COLUMNAS 'adults' Y 'children'
    # ASIGNACIÓN DE MEDIANA A VALORES NULL
    df.loc[df["adults"].isnull(), "adults"] = 2
    df.loc[df["children"].isnull(), "children"] = 0
    df.loc[(df["adults"] < 0), "adults"] = 0
    df.loc[(df["children"] < 0), "children"] = 0
    df.loc[((df["adults"] <= 0)&(df["children"] <= 0)), "adults"] = 2
    #---------------------------- CAMBIO 4-----------------------------------------
    # TRANSFORMACIÓN DE 'children' A CATEGÓRICA BINARIA
    df.loc[df["children"] > 0, "children"] = 1
    #---------------------------- CAMBIO 5-----------------------------------------
    # TRANSFORMACIÓN DE 'total_book' A CATEGÓRICA BINARIA
    # ASIGNACIÓN DE MEDIANA A VALORES NULL
    df.loc[df["total_book"].isnull(), "total_book"] = 0
    df.loc[df["total_book"] < 0, "total_book"] = 0
    df.loc[df["total_book"] > 0, "total_book"] = 1
    #---------------------------- CAMBIO 6-----------------------------------------
    # TRANSFORMACIÓN DE 'pre_cancel' Y 'pre_not_cancel' A CATEGÓRICA BINARIA
    # ASIGNACIÓN DE MEDIANA A VALORES NULL
    df.loc[df["pre_cancel"].isnull(), "pre_cancel"] = 0
    df.loc[df["pre_not_cancel"].isnull(), "pre_not_cancel"] = 0
    df.loc[df["pre_cancel"] < 0, "pre_cancel"] = 0
    df.loc[df["pre_not_cancel"] < 0, "pre_not_cancel"] = 0
    df.loc[df["pre_cancel"] > 0, "pre_cancel"] = 1
    df.loc[df["pre_not_cancel"] > 0, "pre_not_cancel"] = 1
    
    #---------------------------- CAMBIO 8-----------------------------------------
    # ASIGNACIÓN DE MEDIANA EN VALORES INCONSISTENTES COINCIDENTES EN LAS COLUMNAS 'week_nights' y 'weekend_nights'
    # TRANSFORMACIÓN DE ETIQUETAS PARA UNIFICAR LOS VALORES POR ENCIMA DE 5 EN 'week_nights' Y POR ENCIMA DE 2 EN 'weekend_nights'
    # ASIGNACIÓN DE MEDIANA A VALORES NULL
    df.loc[df["week_nights"].isnull(), "week_nights"] = 2
    df.loc[df["weekend_nights"].isnull(), "weekend_nights"] = 1
    df.loc[df["week_nights"] < 0, "week_nights"] = 0
    df.loc[df["weekend_nights"] < 0, "weekend_nights"] = 0
    df.loc[((df["week_nights"] == 0)&(df["weekend_nights"] == 0)), "week_nights"] = 2    
    df.loc[df["week_nights"] > 5, "week_nights"] = 6
    df.loc[df["weekend_nights"] > 2, "weekend_nights"] = 3
    #---------------------------- CAMBIO 9-----------------------------------------
    # TRANSFORMACIÓN DE 'meal_plan' A VARIABLE NUMÉRICA CON ORDINAL ENCODER
    # ASIGNACIÓN DE MODA A VALORES NULL
    list_meal = ["SC","RO","BB","HB","FB"]
    df.loc[df["meal_plan"].isnull(), "meal_plan"] = "BB"
    df.loc[~df["meal_plan"].isin(list_meal), "meal_plan"] = "BB"   
    encoder = OrdinalEncoder(categories=[["SC","RO","BB","HB","FB"]])    
    df[["meal_plan"]] = encoder.fit_transform(df[["meal_plan"]])
    #---------------------------- CAMBIO 10-----------------------------------------
    # TRANSFORMACIÓN DE 'parking' A VARIABLE BINARIA
    # ASIGNACIÓN DE MEDIANA A VALORES NULL
    df.loc[df["parking"].isnull(), "parking"] = 0
    df.loc[df["parking"] < 0, "parking"] = 0
    df.loc[df["parking"] > 0, "parking"] = 1

    #---------------------------- CAMBIO 7-----------------------------------------
    # ELIMINACIÓN DE 'arr_date'
    df.drop(columns = ["arr_date"], inplace = True)

    return df

In [95]:
X_train_transform = transform_test(df)
X_train_transform

,daily_price,lead_time,adults,children,repeated_guest,total_book,pre_cancel,pre_not_cancel,arr_week,week_nights,weekend_nights,meal_plan,parking,special_req,canceled
0,1.812913,2.350248,2,0,0,0,0,0,40,2,1,2.0,0,0,0
1,2.028083,0.698970,2,0,0,0,0,0,45,3,2,1.0,0,1,0
2,1.778151,0.000000,1,0,0,0,0,0,9,1,2,2.0,0,0,1
3,2.000000,2.324282,2,0,0,0,0,0,20,2,0,2.0,0,0,1
4,1.975432,1.681241,2,0,0,0,0,0,15,1,1,1.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153350,1.982904,1.361728,2,0,0,0,0,0,35,5,2,2.0,0,0,0
153351,2.353012,2.008600,3,0,0,0,0,0,35,5,2,2.0,0,2,0
153352,2.197859,1.531479,2,0,0,0,0,0,35,5,2,2.0,0,4,0
153353,2.018700,2.037426,2,0,0,0,0,0,35,5,2,2.0,0,0,0


In [5]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)
target = "canceled"

In [6]:
# Se forman los subconjuntos X,y
X_train = train_set.copy()
X_train.drop(columns = [target], inplace = True)
y_train = train_set[target].copy()

X_test = test_set.copy()
X_test.drop(columns = [target], inplace = True)
y_test = test_set[target].copy()

In [7]:
# LISTAS
columns_complete = X_train.columns.to_list()
columns_to_include = ['daily_price','lead_time','adults','children','repeated_guest','total_book','pre_cancel','pre_not_cancel','arr_date','arr_week','week_nights','weekend_nights','meal_plan','parking','special_req']
columns_to_exclude = [col for col in columns_complete if col not in columns_to_include]
columns_to_oe = ['meal_plan']

feat_num = X_train.select_dtypes(include=['number']).columns.to_list()
feat_obj = X_train.select_dtypes(include=['object']).columns.to_list()

In [12]:
df_test = X_train.copy()
print(df_test.iloc[0,0])
df_test.iloc[0,0] = np.nan
print(df_test.iloc[0,0])

62.0
nan


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 152355 entries, 0 to 153354
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   daily_price     152355 non-null  float64
 1   lead_time       152355 non-null  int64  
 2   adults          152355 non-null  int64  
 3   children        152355 non-null  int64  
 4   repeated_guest  152355 non-null  int64  
 5   total_book      152355 non-null  int64  
 6   pre_cancel      152355 non-null  int64  
 7   pre_not_cancel  152355 non-null  int64  
 8   arr_date        152355 non-null  object 
 9   arr_week        152355 non-null  int64  
 10  week_nights     152355 non-null  int64  
 11  weekend_nights  152355 non-null  int64  
 12  meal_plan       152355 non-null  float64
 13  parking         152355 non-null  int64  
 14  special_req     152355 non-null  int64  
 15  canceled        152355 non-null  int64  
dtypes: float64(2), int64(13), object(1)
memory usage: 19.8+ MB


In [8]:
transformador = FunctionTransformer(transform_test)

In [ ]:
df_transformado = 

In [478]:
# Seleccionar columnas
#select_step = ColumnTransformer([("Exclude", "drop", columns_to_exclude)], remainder = "passthrough")

# Imputar valores a nulos
imputer_step = ColumnTransformer(
    [("Impute_Median", SimpleImputer(strategy = "median"), feat_num),
     ("Impute_Mode", SimpleImputer(strategy = "most_frequent"), feat_obj)],
     remainder = "passthrough")

# Aplicar OrdinalEncoder
ordinal_step = ColumnTransformer([("OEncoder", OrdinalEncoder(categories=[["SC","RO","BB","HB","FB"]]),columns_to_oe)], remainder = "passthrough")

In [479]:
preprocessing = ColumnTransformer(
    [#("zero_stage", select_step, columns_to_exclude),
     ("first_stage", imputer_step, columns_to_include),
     ("second_stage", ordinal_step, columns_to_oe)],
     remainder = "passthrough")

In [480]:
preprocessing

ColumnTransformer(remainder='passthrough',
                  transformers=[('first_stage',
                                 ColumnTransformer(remainder='passthrough',
                                                   transformers=[('Impute_Median',
                                                                  SimpleImputer(strategy='median'),
                                                                  ['daily_price',
                                                                   'lead_time',
                                                                   'adults',
                                                                   'children',
                                                                   'repeated_guest',
                                                                   'total_book',
                                                                   'pre_cancel',
                                                                   'pre_not_cancel',
                                                                   'arr_week',
                                                                   'week_nights',
                                                                   'weekend_nights',
                                                                   'parking',
                                                                   'special_req']...
                                 ['daily_price', 'lead_time', 'adults',
                                  'children', 'repeated_guest', 'total_book',
                                  'pre_cancel', 'pre_not_cancel', 'arr_date',
                                  'arr_week', 'week_nights', 'weekend_nights',
                                  'meal_plan', 'parking', 'special_req']),
                                ('second_stage',
                                 ColumnTransformer(remainder='passthrough',
                                                   transformers=[('OEncoder',
                                                                  OrdinalEncoder(categories=[['SC',
                                                                                              'RO',
                                                                                              'BB',
                                                                                              'HB',
                                                                                              'FB']]),
                                                                  ['meal_plan'])]),
                                 ['meal_plan'])])

In [481]:
pipe_preprocessed = preprocessing.fit_transform(X_train)
df_check = pd.DataFrame(pipe_preprocessed, columns= preprocessing.get_feature_names_out())
df_check

,first_stage__Impute_Median__daily_price,first_stage__Impute_Median__lead_time,first_stage__Impute_Median__adults,first_stage__Impute_Median__children,first_stage__Impute_Median__repeated_guest,first_stage__Impute_Median__total_book,first_stage__Impute_Median__pre_cancel,first_stage__Impute_Median__pre_not_cancel,first_stage__Impute_Median__arr_week,first_stage__Impute_Median__week_nights,first_stage__Impute_Median__weekend_nights,first_stage__Impute_Median__parking,first_stage__Impute_Median__special_req,first_stage__Impute_Mode__arr_date,first_stage__Impute_Mode__meal_plan,second_stage__OEncoder__meal_plan
0,62.0,323.0,2.0,0.0,0.0,1.0,1.0,0.0,36.0,1.0,1.0,0.0,0.0,2015-09-05,BB,2.0
1,93.5,29.0,2.0,0.0,0.0,0.0,0.0,0.0,35.0,4.0,0.0,0.0,1.0,2016-08-30,BB,2.0
2,112.5,197.0,2.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,2.0,0.0,0.0,2017-08-12,BB,2.0
3,118.15,213.0,2.0,0.0,0.0,0.0,0.0,0.0,35.0,4.0,0.0,0.0,1.0,2018-08-31,BB,2.0
4,114.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,37.0,1.0,0.0,0.0,1.0,2017-09-16,BB,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121879,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0,0.0,0.0,0.0,2016-04-06,HB,3.0
121880,162.0,151.0,3.0,0.0,0.0,0.0,0.0,0.0,24.0,2.0,1.0,0.0,1.0,2017-06-16,BB,2.0
121881,154.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,39.0,1.0,2.0,0.0,1.0,2016-10-02,BB,2.0
121882,146.0,76.0,2.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,1.0,0.0,0.0,2017-06-12,BB,2.0
